In [1]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque
import time
import tqdm

offene Probleme / mögliche Verbesserungen:
    - in einem Endzustand nicht mehr q(a',s') berechnen
    - Minibatch mit TF erzeugen
    - store old network weights and use them in loss function

In [2]:
def build_dense_model():
    mlp_action_val = tf.keras.models.Sequential()
    mlp_action_val.add(tf.keras.layers.Dense(50, activation='relu'))
    mlp_action_val.add(tf.keras.layers.Dense(50, activation='relu'))
    mlp_action_val.add(tf.keras.layers.Dense(n_acts, activation=None))
    return mlp_action_val

In [6]:
# build graph
#env = gym.make('LunarLander-v2')
env = gym.make('CartPole-v0')

obs_dim = env.observation_space.shape[0]
n_acts = env.action_space.n
gamma = 0.9

obs_ph = tf.placeholder(shape=(None,obs_dim), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None), dtype=tf.int32)
rew_ph = tf.placeholder(shape=(None), dtype=tf.float32)
new_obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
terminal_ph = tf.placeholder(shape=(None), dtype=tf.float32)

# make core of state-action-value function network
mlp_target = build_dense_model()
mlp_action_val = build_dense_model()

# define state action values
old_state_action_values = mlp_action_val(obs_ph)
new_state_action_values = mlp_target(new_obs_ph)

# update = update_target_network(model, target_model)

# select action
greedy_action = tf.math.argmax(old_state_action_values, axis=1)

# define loss function
y = rew_ph + gamma * tf.reduce_max(new_state_action_values, axis=1)*(1-terminal_ph)
y_no_grad = tf.stop_gradient(y) 
action_masks = tf.one_hot(act_ph, n_acts)
old_selected_action_values = tf.reduce_sum(action_masks * old_state_action_values, axis=1)
action_value_loss = tf.losses.mean_squared_error(y_no_grad, old_selected_action_values)

# define optimizer
optimizer_action_value = tf.train.AdamOptimizer(0.001)
train_action_value = optimizer_action_value.minimize(action_value_loss)
gradients = optimizer_action_value.compute_gradients(action_value_loss)

In [7]:
def dqn(sess, env, experience_replay_buffer, n_episodes):
    global mlp_action_val, mlp_target, new_obs_ph, new_state_action_values
    row_pointer = 0
    learnable = False
    for i in range(n_episodes):
        if i % 100 == 0:
            sess.run([v_t.assign(v) for v_t, v in zip(mlp_target.trainable_weights, mlp_action_val.trainable_weights)])
        done = False
        state = env.reset()
        cnt = 0
        while not done:
            action = select_action(sess, state, 0.1)
            new_state, reward, done, _ = env.step(action)
            add_sample(experience_replay_buffer, state, action, reward, new_state, done, row_pointer)
            row_pointer += 1
            if row_pointer == exp_replay_size:
                learnable = True
            row_pointer %= exp_replay_size
            state = new_state
            if learnable:
                update_q_function()
            cnt += 1

        if i % 100 == 0:
            print('Evaluation after',i,'episodes:',evaluate(sess, env, 100))

            
def add_sample(experience_replay_buffer, state, action, reward, new_state, done, row_pointer):
    experience_replay_buffer[row_pointer, :obs_dim] = state
    experience_replay_buffer[row_pointer, obs_dim:2*obs_dim] = new_state
    experience_replay_buffer[row_pointer, 2*obs_dim] = action
    experience_replay_buffer[row_pointer, 2*obs_dim+1] = reward
    experience_replay_buffer[row_pointer, 2*obs_dim+2] = float(done)
    
def evaluate(sess, env, iterations):
    sum_return = 0
    for i in range(iterations):
        done = False
        state = env.reset()
        while not done:
#             if i == 0:
#                 env.render()
            action = select_action(sess, state, 0)
            new_state, reward, done, _ = env.step(action)
            state = new_state
            sum_return += reward
        env.close()
    return sum_return / iterations
            

        
def update_q_function():
    experience_batch = sample_experience_batch()
    states, actions, rewards, new_states, terminal_flags = extract_data(experience_batch)
    loss = gradient_step(states, actions, rewards, new_states, terminal_flags)
    return loss

    
def gradient_step(states, actions, rewards, new_states, terminal_flags):
    _, loss, debug_output = sess.run([train_action_value, action_value_loss, y], feed_dict={
                                obs_ph: np.array(states).reshape(-1, obs_dim),
                                act_ph: np.array(actions),
                                rew_ph: np.array(rewards),
                                new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
                                terminal_ph: np.array(terminal_flags)
                             })
    return loss
    
def extract_data(experience_batch):
    states = [x[:obs_dim] for x in experience_batch]
    actions = [x[2*obs_dim] for x in experience_batch]
    rewards = [x[2*obs_dim+1] for x in experience_batch]
    new_states = [x[obs_dim:2*obs_dim] for x in experience_batch]
    terminal_flags = [x[2*obs_dim+2] for x in experience_batch]
    return states, actions, rewards, new_states, terminal_flags
    
def sample_experience_batch(batch_size=32):
    indices = np.random.choice(np.arange(len(experience_replay_buffer)), batch_size)
    sample_batch = experience_replay_buffer[indices]
    return sample_batch

def select_action(sess, state, eps):
    if np.random.rand() < eps:
        action = np.random.randint(n_acts)
    else:
        action = sess.run(greedy_action, {obs_ph: state.reshape(1,-1)})[0]
    return action

In [8]:
# main
exp_replay_size = 500
memory_width = 3 + 2*obs_dim
experience_replay_buffer = np.zeros((exp_replay_size, memory_width))
sess = tf.Session()
sess.run(tf.global_variables_initializer())
starttime = time.time()
dqn(sess, env, experience_replay_buffer, 10000)
stoptime = time.time()
print('Overall time needed:',stoptime-starttime)

Evaluation after 0 episodes: 10.46
Evaluation after 100 episodes: 9.35
Evaluation after 200 episodes: 9.43
Evaluation after 300 episodes: 9.51
Evaluation after 400 episodes: 9.64
Evaluation after 500 episodes: 43.68
Evaluation after 600 episodes: 134.73
Evaluation after 700 episodes: 179.91
Evaluation after 800 episodes: 179.28
Evaluation after 900 episodes: 141.46
Evaluation after 1000 episodes: 181.85
Evaluation after 1100 episodes: 164.3
Evaluation after 1200 episodes: 200.0
Evaluation after 1300 episodes: 200.0
Evaluation after 1400 episodes: 138.87
Evaluation after 1500 episodes: 13.03
Evaluation after 1600 episodes: 165.35


KeyboardInterrupt: 